In [12]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
!pip install --upgrade mxnet-cu100
!pip install autogluon
!pip install -U ipykernel

--2020-05-20 08:44:45--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2571184 (2.5M) [text/plain]
Saving to: ‘doctor_train.csv.1’

doctor_train.csv.1  100%[===================>]   2.45M  --.-KB/s    in 0.09s   

2020-05-20 08:44:46 (26.1 MB/s) - ‘doctor_train.csv.1’ saved [2571184/2571184]

--2020-05-20 08:44:49--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awa

     |████████████████████████████████| 122kB 8.7MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.2.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 0.25.3 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


# RESTART RUNTIME TO SEE CHANGES!

In [0]:
import autogluon as ag
from autogluon import TabularPrediction as task
import time
import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
test_df = pd.read_csv('doctor_test.csv')

cols = ['age','Money','day','Time','Doctor_visits','last_visit',
        'cured_in','Profession','Status','edu','Irregular','residence',
        'prev_diagnosed','communication','Month','side_effects']
dep_var = ['Y']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

df_D = pd.get_dummies(df[cols])
test_df_D = pd.get_dummies(test_df[cols])
df_D=df_D.fillna(0)
# df_D[df_D < 0] = 0
test_df_D=test_df_D.fillna(0)
# test_df_D[test_df_D < 0] = 0
df_D['Y'] = df['Y']

In [7]:
predictor = task.fit(train_data=df_D, 
                     label='Y', 
                     eval_metric='accuracy',
                     auto_stack=True,
                     verbosity=2)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200520_091324/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200520_091324/
Train Data Rows:    28196
Train Data Columns: 52
Preprocessing data ...
Here are the first 10 unique label values in your data:  [0 1]
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: [('binary', 'multiclass', 'regression')])

Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Feature Generator processed 28196 data points with 51 features
Original Features:
	float features: 2
	int features: 49
	object features: 0
Generated Features:
	int features: 0
All Features:
	float features: 2
	int features: 49
	object features: 0
	Data preprocessing and feature engineering runtime = 0.21s ...
AutoGluon will gauge predictive performanc

In [8]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                                    model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer
0           CatboostClassifier_STACKER_l1   0.914385      20.440711  1301.808281                0.130006          20.754608            1       True
1                 weighted_ensemble_k0_l2   0.914385      20.453127  1313.541078                0.012416          11.732797            2       True
2     LightGBMClassifierCustom_STACKER_l1   0.913427      20.623550  1321.597962                0.312845          40.544289            1       True
3           LightGBMClassifier_STACKER_l1   0.913037      20.495679  1298.936280                0.184974          17.882607            1       True
4                 weighted_ensemble_k0_l1   0.912115       0.389413    67.163117                0.012268          11.820253            1       True
5   RandomForestClassifierEntr_STACKER_l1   0.9121

In [11]:
%%time
y_hat = predictor.predict(test_df_D)

CPU times: user 41 s, sys: 7.43 s, total: 48.5 s
Wall time: 2min 37s


In [0]:
le_y = le.inverse_transform(y_hat)

output = pd.DataFrame({'id': test_df.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')